In [36]:
import numpy as np
import pandas as pd

from numpy.linalg import inv

# Loading Data_Frame
data_x_df = pd.read_csv('data_X.csv')
data_t_df = pd.read_csv('data_T.csv')
data_df = pd.merge(data_x_df, data_t_df, left_index=True, right_index=True)

sizeData = data_df.shape[0]
sizeFeature = data_df.shape[1]

# Create Dataset
rawData = np.array(data_df)

# Random Shuffle for Raw Data
np.random.shuffle(rawData)

# Split Dataset into Train & Valid set
trainData = rawData[:int(sizeData*90/100)]
valData = rawData[int((-sizeData)*10/100):]

sizeTrain = trainData.shape[0]
sizeVal = valData.shape[0]

In [37]:
"""
Function:
- Create Phi(x) for M=1 & M=2

Parameters:
- train: trainData[:, :-1]
"""
def createPhi(train):
    len = train.shape[1]
    
    # M >= 1
    phiOne = np.insert(train, 0, values=np.ones(train.shape[0]), axis=1)
    
    # M >= 2
    phiTwo = phiOne
    diff = 0
    offset = len + 1;
    for i in range(len):
        for j in range(len):
            if i <= j:
                offset += 1
                phiTwo = np.insert(phiTwo, 
                                  i*len + j + len + 1 - diff,
                                  values = train[:, i]*train[:, j],
                                  axis = 1)
            else:
                diff += 1
    # M >= 3
    diff = 0
    phiThree = phiTwo
    for i in range(len):
        for j in range(len):
            for k in range(len):
                if i <= j and j <= k:
                    phiThree = np.insert(phiThree,
                                        i*len*len + j*len + k + offset - diff,
                                        values = train[:, i]*train[:, j]*train[:, k],
                                        axis = 1)
                else:
                    diff += 1

    return phiOne, phiTwo, phiThree

In [38]:
"""
Function:
- Create y(x,w) as Prediction function

Parameters:
- inputX : trainData[:, :-1]
- inputT : trainData[:, -1]

Return:
- Root Mean Square Value
"""
def prediction(inputX, inputT, M, weight):
    len = inputX.shape[1] #8
    y = np.zeros(inputX.shape[0])
    y += weight[0]
    
    # M >= 1 
    for i in range(len):
        m1 = np.multiply(inputX[:, i], weight[i+1])
        y += m1
    
    # M >= 2 with interaction terms
    offset = len + 1;
    if M >= 2:
        diff = 0
        for i in range(len):
            for j in range(len):
                if i <= j:
                    offset += 1
                    m2 = np.multiply(inputX[:, i], inputX[:, j])
                    y += np.multiply(m2, weight[i*len + j + len + 1 - diff])
                else:
                    diff += 1
    
    # M >= 3 with interaction terms (for (2-b))
    if M >= 3:
        diff = 0
        for i in range(len):
          for j in range(len):
              for k in range(len):
                  if i <= j and j <= k:
                      m3 = np.multiply(np.multiply(inputX[:, i], inputX[:, j]), inputX[:, k])
                      y += np.multiply(m3, weight[i*len*len + j*len + k + offset - diff])                    
                  else:
                      diff += 1
    
    return np.sqrt(np.mean((y - inputT)**2)) 

In [39]:
# Feature Selection - (a)
phiOne, phiTwo, _ = createPhi(trainData[..., :-1])

# perform Least Square solution
"""
(X.T*X)-1 * X.T * Y
"""
weightM1 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(phiOne.T, phiOne)),
                      phiOne.T
                  ),
                  trainData[:, -1]) # Target
weightM2 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(phiTwo.T, phiTwo)),
                      phiTwo.T
                  ),
                  trainData[:, -1]) # Target

train_MLE_RMS = [prediction(trainData[..., :-1], trainData[..., -1], 1, weightM1), 
            prediction(trainData[..., :-1], trainData[..., -1], 2, weightM2)]
val_MLE_RMS = [prediction(valData[..., :-1], valData[..., -1], 1, weightM1), 
            prediction(valData[..., :-1], valData[..., -1], 2, weightM2)]


In [40]:
print("===Feature Selection - (a)===\n")
print("Weight : \n")
print("M = 1 : \n", weightM1)
print("M = 2 : \n", weightM2, '\n')

print("Trainning set's RMS result:")
print("M = 1  ", train_MLE_RMS[0])
print("M = 2  ", train_MLE_RMS[1], '\n')

print("Valid set's RMS result:")
print("M = 1  ", val_MLE_RMS[0])
print("M = 2  ", val_MLE_RMS[1], '\n')

===Feature Selection - (a)===

Weight : 

M = 1 : 
 [-3.60904109e+06 -4.29905734e+04 -4.27257971e+04  1.15824952e+03
 -8.12307821e+00  1.16523115e+02 -3.73516536e+01  4.20719286e+01
  4.02625928e+04]
M = 2 : 
 [ 7.07313286e-01  1.33908244e+02 -4.36977990e+01 -3.13917693e+01
 -6.25494223e+02 -2.47491682e+02 -1.75578351e+02 -1.41007836e+02
  2.43588580e+01  3.26493288e+01  2.04228872e+02 -2.75549531e+01
 -6.23998414e+00 -7.86675495e+00 -2.47546488e-01 -5.39105708e+00
 -9.08563744e+02  3.23911790e+02 -1.29283457e+02 -4.84691360e+00
 -7.54384210e+00  4.82273640e+00 -2.09195367e+01 -1.39097373e+03
  2.69861853e+01 -5.50037072e-01  2.49793535e+00 -1.63749410e+00
  5.11040726e+00  2.63331348e+02 -1.97784014e-03  4.71217723e-02
 -3.27899560e-03 -9.13028129e-03  8.59841851e+00 -2.15317243e-01
  1.33384838e-02  9.74260707e-02 -5.09446259e+01  3.59065058e-03
 -2.40551442e-02 -8.06212160e+00  2.05003519e-02  3.24610701e+01
 -2.33700717e+03] 

Trainning set's RMS result:
M = 1   69506.21518118356
M

In [41]:
# Feature Selection - (b)

deleteOneOfFeature = [np.delete(trainData, i, 1) for i in range(trainData.shape[1]-1)]

phiOneArray = []
for i in range(trainData.shape[1]-1):
    _phiOne, _ , _= createPhi(deleteOneOfFeature[i][...,:-1])
    phiOneArray.append(_phiOne)

weightMLE1Array = []
for i in range(trainData.shape[1]-1):
    _weightM1 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(phiOneArray[i].T, phiOneArray[i])),
                      phiOneArray[i].T
                  ),
                  deleteOneOfFeature[i][..., -1]) # Target
    weightMLE1Array.append(_weightM1)          
      
train_MLE_RMS_Array = []
for i in range(trainData.shape[1]-1):
    _trainRMS = prediction(deleteOneOfFeature[i][..., :-1], deleteOneOfFeature[i][..., -1], 1, weightMLE1Array[i])
    train_MLE_RMS_Array.append(_trainRMS)

In [42]:
print("===Feature Selection - (b)===\n")

for i in range(len(train_MLE_RMS_Array)):
    print("Delete No.", i+1, "Feature => RMS would be : ",train_MLE_RMS_Array[i])

print("\n\nWe can find that the final RMS is the most,")
print("which means without the median_income, the final predict would be the most unreliable.\n")
print("[Median_income] is the most contributive feature!!")

===Feature Selection - (b)===

Delete No. 1 Feature => RMS would be :  75378.9699778046
Delete No. 2 Feature => RMS would be :  75981.25850956484
Delete No. 3 Feature => RMS would be :  70710.1716584494
Delete No. 4 Feature => RMS would be :  69685.33428453187
Delete No. 5 Feature => RMS would be :  69983.26355475682
Delete No. 6 Feature => RMS would be :  71552.24533191256
Delete No. 7 Feature => RMS would be :  69558.67572081064
Delete No. 8 Feature => RMS would be :  90840.63228032186


We can find that the final RMS is the most,
which means without the median_income, the final predict would be the most unreliable.

[Median_income] is the most contributive feature!!


In [53]:
# Maximum likelihood approach - (a)
print("===Maximum likelihood approach - (a)===\n")
print('Choose [Polynomial] as the basis function!!')
print('1. Polynomial models have moderate flexibility of shapes.')
print('2. The higher order of Polynomial models can present the interactions between variables.')
print('3. Polynomial models have well known and understood properties.')
print('4. Besides, Polynomial model is easy to implement.')

===Maximum likelihood approach - (a)===

Choose [Polynomial] as the basis function!!
1. Polynomial models have moderate flexibility of shapes.
2. The higher order of Polynomial models can present the interactions between variables.
3. Polynomial models have well known and understood properties.
4. Besides, Polynomial model is easy to implement.


In [44]:
"""
Function:
- Gauss-Jordan elimanation

Parameters:
- matrix

Return:
- inverse matrix
"""

def Gauss_Jordan(M):
    # Applying Gauss Jordan Elimination
    I = np.eye(M.shape[0])
    inv = np.linalg.solve(M, I)
    return inv

In [45]:
# Maximum likelihood approach - (b)

_, _, phiThree = createPhi(trainData[..., :-1])

# perform Least Square solution
# """
# W = (X.T*X)^-1 * X.T * Y
# """

_inv = Gauss_Jordan(np.dot(phiThree.T, phiThree))
weightM3 = np.dot(np.dot(
                      _inv,
                      phiThree.T),
                  trainData[:, -1]) # Target

_train_MLE_RMS = prediction(trainData[..., :-1], trainData[..., -1], 3, weightM3)
_val_MLE_RMS = prediction(valData[..., :-1], valData[..., -1], 3, weightM3)

In [54]:
# Maximum likelihood approach - (b)
print("===Maximum likelihood approach - (b)===\n")

print('As polynomial model become higher degree, the curve may oscillate between exact-fit values.')
print('However, Setting M = 3 is also suitable for this case. (See below)')
print('For the higher order of M, we may see training loss is low but validation loss is high!\n')

print("Trainning set's RMS result:")
print("M = 1  ", train_MLE_RMS[0])
print("M = 2  ", train_MLE_RMS[1])
print("M = 3  ", _train_MLE_RMS, '\n')

print("Valid set's RMS result:")
print("M = 1  ", val_MLE_RMS[0])
print("M = 2  ", val_MLE_RMS[1])
print("M = 3  ", _val_MLE_RMS, '\n')



===Maximum likelihood approach - (b)===

As polynomial model become higher degree, the curve may oscillate between exact-fit values.
However, Setting M = 3 is also suitable for this case. (See below)
For the higher order of M, we may see training loss is low but validation loss is high!

Trainning set's RMS result:
M = 1   69506.21518118356
M = 2   66902.66248693895
M = 3   58511.594878433185 

Valid set's RMS result:
M = 1   70047.83700973312
M = 2   66777.47111162756
M = 3   63580.02301346249 



In [47]:
# Maximum likelihood approach - (c)

# Apply N-fold cross-validation
N_Fold = 100
sizeFold = int(trainData.shape[0]/N_Fold)

for n in range(N_Fold):
    _val = trainData[sizeFold*n:sizeFold*(n+1), :]
    _train = np.concatenate((trainData[0:sizeFold*n, :], trainData[sizeFold*(n+1):, :]))

    _phiOne, _phiTwo, _ = createPhi(_train[..., :-1])

    _wM1 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(_phiOne.T, _phiOne)),
                     _phiOne.T
                  ),
                  _train[:, -1]) # Target
    _wM2 = np.dot(np.dot(
                      np.linalg.pinv(np.dot(_phiTwo.T, _phiTwo)),
                      _phiTwo.T
                  ),
                  _train[:, -1]) # Target
    if n==0:
        _weightM1 = _wM1
        _weightM2 = _wM2
    else:
        _weightM1 += _wM1
        _weightM2 += _wM2
    
_weightM1 = _weightM1/N_Fold
_weightM2 = _weightM2/N_Fold

train_MLE_RMS_N_Fold = [prediction(trainData[..., :-1], trainData[..., -1], 1, _weightM1), 
            prediction(trainData[..., :-1], trainData[..., -1], 2, _weightM2)]
val_MLE_RMS_N_Fold = [prediction(valData[..., :-1], valData[..., -1], 1, _weightM1), 
            prediction(valData[..., :-1], valData[..., -1], 2, _weightM2)]

In [48]:
print("===Maximum likelihood approach - (c)===\n")
print('Selecting two different order to compare the behavior of model when performing N-Fold cross validation,')
print('We can found when test validation set in M = 2, the RMS tilts up,')
print('We can conclude that M = 2 may be too much complex for this scenario,')
print('Since its RMS perform like overfitting,')

print("Trainning set's RMS result with N-fold cross-validation:")
print("M = 1  ", train_MLE_RMS_N_Fold[0])
print("M = 2  ", train_MLE_RMS_N_Fold[1], '\n')

print("Valid set's RMS result with N-fold cross-validation:")
print("M = 1  ", val_MLE_RMS_N_Fold[0])
print("M = 2  ", val_MLE_RMS_N_Fold[1], '\n')

===Maximum likelihood approach - (c)===

Selecting two different order to compare the behavior of model when performing N-Fold cross validation,
We can found when test validation set in M = 2, the RMS tilts up,
We can conclude that M = 2 may be too much complex for this scenario,
Since its RMS perform like overfitting,
Trainning set's RMS result with N-fold cross-validation:
M = 1   69506.2153257171
M = 2   66902.54775727756 

Valid set's RMS result with N-fold cross-validation:
M = 1   70047.5561857142
M = 2   66775.36479322216 



In [49]:
# Maximum A Posterior - (a)
print("===Maximum a  Posterior approach - (a)===\n")
print('The key difference between MLP & MAP approach:')
print('If we have prior information, then MAP approach is the good way.')
print('Since MLE produces model params. most likely to generate the observed data.')
print('While if no such prior probalilty, then MLE is a reasonable approach.')
print('Since MAP is the model params that is most likely given the observed data.')

===Maximum a  Posterior approach - (a)===

The key difference between MLP & MAP approach:
If we have prior information, then MAP approach is the good way.
Since MLE produces model params. most likely to generate the observed data.
While if no such prior probalilty, then MLE is a reasonable approach.
Since MAP is the model params that is most likely given the observed data.


In [50]:
# Maximum A Posterior - (b)

# Apply a regularizer to (2), the approach will be MAP estimation.

N_Fold = 100
sizeFold = int(trainData.shape[0]/N_Fold)
paramLambda = 0.1

for n in range(N_Fold):
    _val = trainData[sizeFold*n:sizeFold*(n+1), :]
    _train = np.concatenate((trainData[0:sizeFold*n, :], trainData[sizeFold*(n+1):, :]))

    # perform Least Square solution
    # """
    # W = (lambdaI+ X.T*X)^-1 * X.T * Y
    # """ 
    _phiOne, _phiTwo, _ = createPhi(_train[..., :-1])

    _wMAP1 = np.dot(np.dot(
                      np.linalg.pinv((paramLambda* np.eye(_phiOne.shape[1]) + np.dot(_phiOne.T, _phiOne))),
                     _phiOne.T
                  ),
                  _train[:, -1]) # Target
    _wMAP2 = np.dot(np.dot(
                      np.linalg.pinv((paramLambda* np.eye(_phiTwo.shape[1]) + np.dot(_phiTwo.T, _phiTwo))),
                      _phiTwo.T
                  ),
                  _train[:, -1]) # Target
    if n==0:
        _weightMAP1 = _wMAP1
        _weightMAP2 = _wMAP2
    else:
        _weightMAP1 += _wMAP1
        _weightMAP2 += _wMAP2
    
_weightMAP1 = _weightMAP1/N_Fold
_weightMAP2 = _weightMAP2/N_Fold

train_MAP_RMS_N_Fold = [prediction(trainData[..., :-1], trainData[..., -1], 1, _weightMAP1), 
            prediction(trainData[..., :-1], trainData[..., -1], 2, _weightMAP2)]
val_MAP_RMS_N_Fold = [prediction(valData[..., :-1], valData[..., -1], 1, _weightMAP1), 
            prediction(valData[..., :-1], valData[..., -1], 2, _weightMAP2)]

In [51]:
print("===Maximum a  Posterior approach - (b)===\n")

print("Trainning set's RMS result with N-fold cross-validation:")
print("M = 1  ", train_MAP_RMS_N_Fold[0])
print("M = 2  ", train_MAP_RMS_N_Fold[1], '\n')

print("Valid set's RMS result with N-fold cross-validation:")
print("M = 1  ", val_MAP_RMS_N_Fold[0])
print("M = 2  ", val_MAP_RMS_N_Fold[1], '\n')

===Maximum a  Posterior approach - (b)===

Trainning set's RMS result with N-fold cross-validation:
M = 1   69545.83395221547
M = 2   66902.50002653379 

Valid set's RMS result with N-fold cross-validation:
M = 1   70024.6751450012
M = 2   66775.29333980981 



In [52]:
print("===Maximum a  Posterior approach - (c)===\n")
print('The approach difference is presented above.')
print("But compare with MLE result, show quite similar.")
print("Something I have to say is that MAP is still a bit worse than MLE in this case.")
print("If data is big enough, MLE and MAP will converge to same value")
print("When data is less, the hyper parameter we choose may influence results a lot.")


===Maximum a  Posterior approach - (c)===

The approach difference is presented above.
But compare with MLE result, show quite similar.
Something I have to say is that MAP is still a bit worse than MLE in this case.
If data is big enough, MLE and MAP will converge to same value
When data is less, the hyper parameter we choose may influencet results a lot.
